In [117]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
ZATOSHI = 100000000
CHART = False

In [118]:
# Select Dataset here to analyze 
# data = pd.read_csv("data/testnet-data.txt")

# Read in mainnet data
data = pd.read_csv("data/mainnet.csv")

#remove last day (partial data)
data = data[:-1]

In [120]:
def calctype(row):
    jsin = row["totalJSIn"] != 0.
    jsout = row["totalJSOut"] != 0.
    vin = row["totalVIn"] != 0.
    vout = row["totalVOut"] != 0.
    
    if row["isCoinBase"] == "T":
        t = "coingen"
    elif vin and vout and not jsin and not jsout:
        t = "unshielded"
    elif vin and jsout and not vout and not jsin:
        t = "shielding"
    elif vout and jsin and not vin and not jsout:
        t = "unshielding"
    elif not vin and not vout:
        t = "shielded"
    else:
        t = "mixed"
        
    return t

In [121]:
# Load and gather data into groups
# Clean and format data
data["blockDate"] = pd.to_datetime(data['blockTime'], unit='s').dt.date
date_group = data.groupby(['blockDate'])

daily_totals = date_group.sum()
cumulative_totals = daily_totals.cumsum()

#coinbase group and daily
coinbase_data = data[data["isCoinBase"] == 'T']
coinbase_daily = coinbase_data.groupby(['blockDate']).sum().cumsum()

#Non Coinbase group and calc fees
non_coinbase = data[data["isCoinBase"] == "F"]
daily_fees = non_coinbase.groupby(['blockDate']).sum().cumsum()
daily_fees["fees"] = daily_fees["totalVIn"] - daily_fees["totalVOut"] + daily_fees["totalJSIn"] - daily_fees["totalJSOut"]

#Add coinbase column to cumulative totals
cumulative_totals["coinbase"] = coinbase_daily['totalVOut'] - daily_fees["fees"]

In [122]:
# Calculate types
data["type"] = data.apply(calctype, axis=1)

In [168]:
# group by transaction type and day
type_group = data.groupby(["blockDate", "type"])

#count the number of each type per day
daily_types = type_group.count().unstack()["blockHeight"]

# Clean data and get total
daily_types.fillna(value=0, inplace=True)
daily_tx_totals = daily_types.sum(axis=1)

# Get the cumulative daily totals
cumulative_daily_tx_totals = daily_tx_totals.cumsum()
cumulative_daily_tx_totals.fillna(value=0, inplace=True)

# Calculate Cumulative types
cumulative_types = daily_types.cumsum()

cumulative_percentage_types = cumulative_types.div(cumulative_daily_tx_totals, axis=0)
daily_percentage_types = daily_types.div(daily_tx_totals, axis=0)

In [164]:
# Calculate various output types
cumulative_totals["shieldedAmount"] = cumulative_totals["totalJSOut"]-cumulative_totals["totalJSIn"]
cumulative_totals["transparentAmount"] = cumulative_totals["totalVOut"]-cumulative_totals["totalVIn"]

cumulative_totals["transparentPercentage"] = cumulative_totals["transparentAmount"]/cumulative_totals["coinbase"]
cumulative_totals["shieldedPercentage"] = cumulative_totals["shieldedAmount"]/cumulative_totals["coinbase"]

cumulative_totals["calc_total"] = cumulative_totals["transparentAmount"] + cumulative_totals["shieldedAmount"]

coinbase_daily["totalVOut"] = coinbase_daily["totalVOut"]/ZATOSHI

In [165]:
# Total TX volume per day (by number of txs)
total_daily_tx_count = data.groupby(["blockDate"]).count()
total_daily_tx_count = total_daily_tx_count.iloc[:,0]
total_daily_tx_count.to_frame()
total_daily_tx_count.columns = ["num_transactions"]

In [166]:
# Calculate coinbase funds (total circulating funds)
coinbase_transactions = data[data["isCoinBase"] == True].groupby(['blockDate'])
coinbase_sum = coinbase_transactions.sum()

In [167]:
#Generate Output
(cumulative_percentage_types*100).to_csv("output/cumulative_tx_types_percentage.csv")
(daily_percentage_types*100).to_csv("output/daily_tx_types_percentage.csv")
cumulative_types.to_csv("output/cumulative_tx_types.csv")
cumulative_totals[['transparentPercentage', 'shieldedPercentage']].to_csv("output/transparent_vs_shielded_percentage.csv")
cumulative_totals['shieldedAmount'].to_csv("output/shielded_amount.csv")
total_daily_tx_count = total_daily_tx_count.to_frame()

total_daily_tx_count.columns = ["Transactions"]
total_daily_tx_count[:-1].to_csv("output/daily_tx_total.csv")